# Making sparse predictions, and forecasting the requests of the government declaration of natural disaster for a drought event in France
In this notebook, we will show how to predict the cost of drought event.
The methodology is presented in the paper [Making sparse predictions, and forecasting the requests of the government declaration of natural disaster for a drought event in France]() by 
T. T. Y. Nguyen,G. Ecoto and A. Chambaz. 

## Imports and installs

In [2]:
import numpy as np
import torch
import pandas as pd
from predicters import OTpreds
from utils import *
import logging
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE


logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
torch.set_default_tensor_type('torch.DoubleTensor')

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


# Example 1: synthesic data

We now present an illustration based on simulated data.

## Data loading
The dataset loader utilities assume there is a "simulations/" folder in the current directory.


In [8]:
file = './simulations/data_simulate.npz'
data = np.load(file, allow_pickle = True)

train = data['train']
test = data['test']
theta_SL_1 = data['theta_SL1']
theta_SL_2 = data['theta_SL2']
rmse_SL1 = data['rmse_SL1']
rmse_SL2 = data['rmse_SL2']

rmse_sl1 = np.mean(rmse_SL1)
rmse_sl2 = np.mean(rmse_SL2)


nb = 0
theta_sl1 = theta_SL_1[nb].squeeze()
theta_sl2 = theta_SL_2[nb].squeeze()
z = train[nb]
y = train[nb][:,2]

### Load the data and whiten it

In [9]:
zp_true = test[nb]
theta_true = zp_true[:, -1]
zp = zp_true.copy()
zp[:,-1] = np.nan
tau_true = np.mean(theta_true)
zp = zp_true.copy()
zp[:,-1] = np.nan

Z = torch.from_numpy(z)
Zp = torch.from_numpy(zp)
Zp_true = torch.from_numpy(zp_true)
Theta_true = torch.from_numpy(theta_true)



### Hyperparameters

In [10]:
alpha = 500
init = theta_sl2
tau = np.mean(init)
cost = dissim2(alpha).cost2

batchsize = 128
eps = 1e-2
lr = 1e-3
mu = 1e-4
report_interval = 200
niter = 2000


### Optimal transport sparse prediction

In [11]:
sk_imputer = OTpreds(n_pairs = 1, noise = 0.01, batchsize = 128, niter = niter, eps = eps, lr = lr, 
                     mu = mu, tau = tau, cost = cost)

theta_ot, maes, rmses, aucs = sk_imputer.fit_transform_update(Z, Zp, theta_true = Theta_true,
                                                           init = None, verbose=True, 
                                                           report_interval=report_interval)



INFO:root:batchsize = 128, epsilon = 0.0100
INFO:root:Iteration, learning rate 0:	 Loss: 35.3213	 Validation MAE: 0.1368	RMSE: 0.2274	AUC: 0.5852
INFO:root:Iteration, learning rate 200:	 Loss: 72.1279	 Validation MAE: 0.0922	RMSE: 0.2059	AUC: 0.9230
INFO:root:Iteration, learning rate 400:	 Loss: 49.1115	 Validation MAE: 0.0830	RMSE: 0.1983	AUC: 0.9301
INFO:root:Iteration, learning rate 600:	 Loss: 40.7352	 Validation MAE: 0.0804	RMSE: 0.1932	AUC: 0.9310
INFO:root:Iteration, learning rate 800:	 Loss: 54.7717	 Validation MAE: 0.0793	RMSE: 0.1881	AUC: 0.9329
INFO:root:Iteration, learning rate 1000:	 Loss: 47.2669	 Validation MAE: 0.0791	RMSE: 0.1847	AUC: 0.9332
INFO:root:Iteration, learning rate 1200:	 Loss: 54.5402	 Validation MAE: 0.0782	RMSE: 0.1817	AUC: 0.9328
INFO:root:Iteration, learning rate 1400:	 Loss: 53.6005	 Validation MAE: 0.0774	RMSE: 0.1787	AUC: 0.9398
INFO:root:Iteration, learning rate 1600:	 Loss: 88.5081	 Validation MAE: 0.0771	RMSE: 0.1773	AUC: 0.9320
INFO:root:Iteratio

In [14]:

theta_hybrid = np.sqrt(theta_sl2*theta_ot)



mse_sl2 = MSE(theta_true, theta_sl2)
mse_hybrid = MSE(theta_true, theta_hybrid)
mse_ot = MSE(theta_true, theta_ot)


print('MSE:\nOT prediction: {:.4f}'.format(mse_ot), '\t SL2 prediction: {:.4f}'.format(mse_sl2), 
      '\t hybrid prediction: {:.4f}'.format(mse_hybrid))

theta_ot_np = theta_ot>np.quantile(theta_ot, 1-tau_true)
theta_sl2_np = theta_sl2> np.quantile(theta_sl2, 1-tau_true)
theta_hybrid_np = theta_hybrid>np.quantile(theta_hybrid, 1-tau_true)
print('AUC:\nbinary OT prediction: {:.2f}'.format(accuracy_score(theta_true, theta_ot_np)), '\t binary SL2 prediction: {:.2f}'.format(accuracy_score(theta_true, theta_sl2_np)), 
      '\t binary hybrid prediction: {:.2f}'.format(accuracy_score(theta_true, theta_hybrid_np)))

print('F1-score:\nbinary OT prediction: {:.2f}'.format(f1_score(theta_true, theta_ot_np)), '\t binary SL2 prediction: {:.2f}'.format(f1_score(theta_true, theta_sl2_np)), 
      '\t binary hybrid prediction: {:.2f}'.format(f1_score(theta_true, theta_hybrid_np)))


MSE:
OT prediction: 0.0308 	 SL2 prediction: 0.0307 	 hybrid prediction: 0.0296
AUC:
binary OT prediction: 0.96 	 binary SL2 prediction: 0.96 	 binary hybrid prediction: 0.96
F1-score:
binary OT prediction: 0.60 	 binary SL2 prediction: 0.62 	 binary hybrid prediction: 0.64


# Example 2: Real data 
## Optimal transport-based prediction

### Data loading

In [ ]:
fil_data = "dat_new.csv"
D = pd.read_csv(fil_data, header=0)
column = list(D.columns)

In [ ]:
var_id = ['INSEE_COM', 'INSEE_DEP']
var_1 = ['Nb_sinistres_voisins', 'Nb_demandes_voisins', 'Nb_primo_demanderesses', 'Prop_sinistres_voisins', 
         'taux_demandes_voisins', 'Nb_primo_demanderesses_5_ans', 'taux_demandes_voisins_5_ans', 'Nb_sinistres_DEPT',
         'Prop_sinistres_DEPT', 'Nb_primo_demanderesses_DEPT', 'Nb_primo_demanderesses_5_ans_DEPT', 
         'taux_demandes_DEPT', 'taux_demandes_5_ans_DEPT']
var_2 = ['date_valeur', 'Stock_demandes', 'nb_demandes', 'nb_reconnaissance', 'Refus_5_ans', 'Refus_1_ans', 
         'Refus_2_ans', 'nb_demandes_5_ans', 'nb_recos_5_ans', 'taux_demandes', 
         'rapport_demande_temps', "taux_demandes_5_ans" ]
var_3 = ['POP', 'VA', 'nb_maisons', 'SUPERFICIE', 'Z_MOYEN', 'densite_maisons', 'zone_sismique1', 'zone_sismique2', 
       'zone_sismique3', 'zone_sismique4', 'MAISONS_ALEA0', 'MAISONS_ALEA1', 'MAISONS_ALEA2', 'MAISONS_ALEA3', 
       'Age1', 'Age2', 'Age3', 'Age4', 'zone_climatique_1', 'zone_climatique_2', 'zone_climatique_3', 
       'zone_climatique_4', 'zone_climatique_5']
var_4 =['MEAN_SWI', 'MIN_SWI', 'MEAN_SWI_T2T3', 'rang_T1', 'rang_T2', 'rang_T3', 'rang_T4', 'rang_m1', 'rang_m2',
        'rang_m3', 'rang_m4', 'rang_m5', 'rang_m6', 'rang_m7', 'rang_m8', 'rang_m9', 'rang_m10', 'rang_m11', 
        'rang_m12', 'MEAN_SWI_T1', 'MEAN_SWI_T2', 'MEAN_SWI_T3', 'MEAN_SWI_T4', 'MEAN_SWI_T1T2T3', 'Eligibilite']

var_5 = ['Exercice']
var_pred = ['preds_Glm', 'preds_cart', 'preds_RF']
var_label = ['Demande']
var_keep = var_1 + var_2 + var_3 + var_4 + var_5 + var_label
var_keep_2 = var_1 + var_2 + var_3 + var_4 + var_label
var_norm = var_1 + var_2 + var_3 + var_4 

### Load the data and whiten it

In [ ]:
DA = D.loc[:,var_keep]

wk = 1
dat_z = DA[DA["Exercice"] != 2021].loc[:,var_keep_2]
dat_21 = DA[DA["Exercice"] == 2021]
weeks = list(np.sort(dat_21['date_valeur'].unique()))
dat_zp = dat_21[dat_21['date_valeur']== weeks[wk]].loc[:,var_keep_2]
z = dat_z.values
zp_true = dat_zp.values
theta_true = zp_true[:, -1]
tau_true = sum(theta_true)/len(theta_true)
zp = zp_true.copy()

zp[:,-1] = np.nan
Z = torch.from_numpy(z)
Zp = torch.from_numpy(zp)
Zp_true = torch.from_numpy(zp_true)
Theta_true = torch.from_numpy(theta_true)


D_21 = D[D['Exercice'] ==2021]
D_21_w0 = D_21[D_21['date_valeur'] == weeks[wk]]
theta_glm = D_21_w0['preds_Glm'].values



### Hyperparameters

In [ ]:
pi = [ 5.65, 42.95,  5.53, 18.88]
nbw = [len(var_1),  len(var_2), len(var_3), len(var_4)]
w = []
alpha = 0.46
for i in range(len(pi)):
    w += nbw[i]*[pi[i]*1/nbw[i]]
    
cost = dissim(torch.tensor(w), alpha).costs
tau = np.mean(theta_glm)

eps =1e-2
lr= 1e-3
mu = 1e-4


### Optimal transport sparse prediction

In [ ]:
sk_imputer = OTpreds(n_pairs = 1, noise = 0.01, batchsize = 128, niter = 30000, eps = eps, lr= lr, 
                    mu = mu, tau = tau, cost = cost )
theta, maes, rmses, aucs = sk_imputer.fit_transform_update(Z, Zp, theta_true = Theta_true,                                                           
                                                           init = None, verbose=True, report_interval=2000)


In [ ]:

theta_hybrid = np.sqrt(theta_sl2*theta_ot)



mse_sl2 = MSE(theta_true, theta_sl2)
mse_hybrid = MSE(theta_true, theta_hybrid)
mse_ot = MSE(theta_true, theta_ot)


print('MSE:\nOT prediction: {:.4f}'.format(mse_ot), '\t SL2 prediction: {:.4f}'.format(mse_sl2), 
      '\t hybrid prediction: {:.4f}'.format(mse_hybrid))

theta_ot_np = theta_ot>np.quantile(theta_ot, 1-tau_true)
theta_sl2_np = theta_sl2> np.quantile(theta_sl2, 1-tau_true)
theta_hybrid_np = theta_hybrid>np.quantile(theta_hybrid, 1-tau_true)
print('AUC:\nbinary OT prediction: {:.2f}'.format(accuracy_score(theta_true, theta_ot_np)), '\t binary SL2 prediction: {:.2f}'.format(accuracy_score(theta_true, theta_sl2_np)), 
      '\t binary hybrid prediction: {:.2f}'.format(accuracy_score(theta_true, theta_hybrid_np)))

print('F1-score:\nbinary OT prediction: {:.2f}'.format(f1_score(theta_true, theta_ot_np)), '\t binary SL2 prediction: {:.2f}'.format(f1_score(theta_true, theta_sl2_np)), 
      '\t binary hybrid prediction: {:.2f}'.format(f1_score(theta_true, theta_hybrid_np)))
